In [1]:
# Contamos únicamente con 8091 imágenes
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
def GenerarListaImagenes(str_Path):

    np_Archivos = np.empty([0])
    for file in Path(str_Path+'Flicker8k_Dataset').glob('*'):
        # print(file)
        # list_Archivos.append(file.name)
        np_Archivos = np.append(np_Archivos, [file.name])

    # Cantidad de imágenes disponibles
    # len(np_Archivos)

    # Se construye el dataframe con base en el arreglo de campos
    columns = ['id_imagen']
    df = pd.DataFrame(data=np_Archivos, columns=columns)
    df.to_csv(str_Path+'/Preprocessing/lista_imagenes.csv', index=False, header=True)


def FormatearDescripciones(str_Path, str_Archiv_Origen, str_NombreCsv):

    # Recorremos el archivo para eliminar los tabs
    str_Fuente = str_Path + '/Flickr8k_text/' + str_Archiv_Origen
    str_Destino = str_Path + '/Flickr8k_text/' + 'Tmp_' + str_Archiv_Origen
    with open(str_Fuente) as Origen, open(str_Destino, 'w') as Destino:
        for line in Origen:
            Destino.write(line.replace('\t', ' '))

    # Recorremos línea a línea el archivo para encontrar en qué punto se separa
    # el id de imagen del texto
    np_Array = np.empty([0,3])
    with open(str_Destino) as file:
        for line in file:
            nbr_Index = line.find(' ')
            line = line.replace('\n', '')
            line = line.replace(' .', '')
            # print(nbr_Index)
            str_Substr1 = line[0:nbr_Index-2]
            str_Substr2 = line[nbr_Index-1]
            str_Substr3 = line[nbr_Index+1:]
            # print(str_Substr2)
            # list_IdImagenes.append(str_Substr1)
            # list_Descr.append(str_Substr2)
            np_Array = np.append(np_Array, [[str_Substr1, str_Substr2, str_Substr3]], axis=0)

    # Se construye el dataframe con base en el arreglo de campos
    columns = ['id_imagen', 'id_seqq', 'descr']
    df = pd.DataFrame(data=np_Array, columns=columns)
    df.to_csv(str_Path+'/Preprocessing/'+str_NombreCsv, index=False, header=True)

In [3]:
# Formateamos los datos para un análisis de los datos
str_Path = '/Users/Marco/github/Img-Gen/'
GenerarListaImagenes(str_Path)
FormatearDescripciones(str_Path, 'Flickr8k.token.txt', 'Token.csv') # Se tarda un rato en esta función

In [20]:
# A partir de lo anterior, se puede generar este data frame
df_Tokens

,id_imagen,id_seqq,descr
0,1000268201_693b08cb0e.jpg,0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,1,A girl going into a wooden building
2,1000268201_693b08cb0e.jpg,2,A little girl climbing into a wooden playhouse
3,1000268201_693b08cb0e.jpg,3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,4,A little girl in a pink dress going into a woo...
...,...,...,...
40455,997722733_0cb5439472.jpg,0,A man in a pink shirt climbs a rock face
40456,997722733_0cb5439472.jpg,1,A man is rock climbing high in the air
40457,997722733_0cb5439472.jpg,2,A person in a red shirt climbing up a rock fac...
40458,997722733_0cb5439472.jpg,3,A rock climber in a red shirt


In [4]:
# Se toma el dataset generado en la celda anterior
df_Tokens = pd.read_csv(str_Path+'/Preprocessing/Token.csv')

# Nos quedamos únicamente con la primer descipción de cada imagen
df_Reducido = df_Tokens.loc[df_Tokens['id_seqq'] == 0]

# Guardamos este csv reducido a una descripción por imagen
df_Reducido.to_csv(str_Path+'/Preprocessing/Token_Reducido.csv', index=False, header=True)

In [21]:
# Y este es el data frame reducido (mismas columnas, se eliminan filas)
df_Reducido

,id_imagen,id_seqq,descr
0,1000268201_693b08cb0e.jpg,0,A child in a pink dress is climbing up a set o...
5,1001773457_577c3a7d70.jpg,0,A black dog and a spotted dog are fighting
10,1002674143_1b742ab4b8.jpg,0,A little girl covered in paint sits in front o...
15,1003163366_44323f5815.jpg,0,A man lays on a bench while his dog sits by him
20,1007129816_e794419615.jpg,0,A man in an orange hat starring at something
...,...,...,...
40435,990890291_afc72be141.jpg,0,A man does a wheelie on his bicycle on the sid...
40440,99171998_7cc800ceef.jpg,0,A group is sitting around a snowy crevasse
40445,99679241_adc853a5c0.jpg,0,A grey bird stands majestically on a beach whi...
40450,997338199_7343367d7f.jpg,0,A person stands near golden walls


In [14]:
# Del dataset reducido, proponemos los siguientes tipos de sujetos principales:

# Animales
# Personas adultas
# Personas jóvenes

# Animales
df_Dogs = df_Reducido.query('descr.str.contains("dog")')
df_Cats = df_Reducido.query('descr.str.contains("cat")')

# Adultos
df_Men = df_Reducido.query('descr.str.contains("man")')
df_Women = df_Reducido.query('descr.str.contains("woman")')

# Niños
df_Boys = df_Reducido.query('descr.str.contains("boy")')
df_Girls = df_Reducido.query('descr.str.contains("girl")')

In [22]:
df_Dogs.to_csv(str_Path+'/Preprocessing/df_Dogs.csv', index=False, header=True)
df_Cats.to_csv(str_Path+'/Preprocessing/df_Cats.csv', index=False, header=True)
df_Men.to_csv(str_Path+'/Preprocessing/df_Men.csv', index=False, header=True)
df_Women.to_csv(str_Path+'/Preprocessing/df_Women.csv', index=False, header=True)
df_Boys.to_csv(str_Path+'/Preprocessing/df_Boys.csv', index=False, header=True)
df_Girls.to_csv(str_Path+'/Preprocessing/df_Girls.csv', index=False, header=True)

In [ ]:
# A partir de aquí, podemos ver de una manera muy general, qué tan equilibrado está el dataset

In [15]:
print('df_Dogs.shape: ', df_Dogs.shape)
print('df_Cats.shape: ', df_Cats.shape)
print()
print('df_Men.shape: ', df_Men.shape)
print('df_Women.shape: ', df_Women.shape)
print()
print('df_Boys.shape: ', df_Boys.shape)
print('df_Girls.shape: ', df_Girls.shape)

df_Dogs.shape:  (1897, 3)
df_Cats.shape:  (149, 3)

df_Men.shape:  (1839, 3)
df_Women.shape:  (476, 3)

df_Boys.shape:  (1074, 3)
df_Girls.shape:  (922, 3)


In [ ]:
# Como se muestra en la celda anterior, están muy desvalanceados los datos

In [18]:
# Del dataset reducido, proponemos los siguientes colores que podrían aparecer en escena:

# Colores de ropa u objetos:
# White, black, blue, yellow, red, green

# Colores de pelo:
# Brown, blonde

# Colores de cosas
df_White = df_Reducido.query('descr.str.contains("white")')
df_Black = df_Reducido.query('descr.str.contains("black")')
df_Blue = df_Reducido.query('descr.str.contains("blue")')
df_Yellow = df_Reducido.query('descr.str.contains("yellow")')
df_Red = df_Reducido.query('descr.str.contains("red")')
df_Green = df_Reducido.query('descr.str.contains("green")')

# Colores de pelo
df_Brown = df_Reducido.query('descr.str.contains("brown")')
df_Blonde = df_Reducido.query('descr.str.contains("blonde")')

In [23]:
df_White.to_csv(str_Path+'/Preprocessing/df_White.csv', index=False, header=True)
df_Black.to_csv(str_Path+'/Preprocessing/df_Black.csv', index=False, header=True)
df_Blue.to_csv(str_Path+'/Preprocessing/df_Blue.csv', index=False, header=True)
df_Yellow.to_csv(str_Path+'/Preprocessing/df_Yellow.csv', index=False, header=True)
df_Red.to_csv(str_Path+'/Preprocessing/df_Red.csv', index=False, header=True)
df_Green.to_csv(str_Path+'/Preprocessing/df_Green.csv', index=False, header=True)
df_Brown.to_csv(str_Path+'/Preprocessing/df_Brown.csv', index=False, header=True)
df_Blonde.to_csv(str_Path+'/Preprocessing/df_Blonde.csv', index=False, header=True)

In [24]:
print('df_White.shape: ', df_White.shape)
print('df_Black.shape: ', df_Black.shape)
print('df_Blue.shape: ', df_Blue.shape)
print('df_Yellow.shape: ', df_Yellow.shape)
print('df_Red.shape: ', df_Red.shape)
print('df_Green.shape: ', df_Green.shape)
print()
print('df_Brown.shape: ', df_Brown.shape)
print('df_Blonde.shape: ', df_Blonde.shape)

df_White.shape:  (830, 3)
df_Black.shape:  (1016, 3)
df_Blue.shape:  (461, 3)
df_Yellow.shape:  (234, 3)
df_Red.shape:  (730, 3)
df_Green.shape:  (254, 3)

df_Brown.shape:  (756, 3)
df_Blonde.shape:  (76, 3)


In [ ]:
# Los colores de cosas, parecen no estar tan desvalanceados
# Los colores de pelo sí lo están